In [68]:
from keras import backend as K
import time
import cv2
import os
import glob
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
import tqdm

In [3]:
from multiprocessing.dummy import Pool
K.set_image_data_format('channels_first')

In [4]:
from fr_utils import *

In [5]:
from inception_blocks_v2 import *

In [7]:
PADDING = 50
#ready_to_detect_identity = True
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [74]:
FRmodel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 96, 96)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 3, 102, 102)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 48, 48)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 64, 48, 48)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [31]:
def prepare_database():
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob("images/*"):
        identity = os.path.splitext(os.path.basename(file))[0].strip('NISDCC-_6g')
        print (identity)
        database[identity] = img_path_to_encoding(file, FRmodel)

    return database

In [33]:
train_images_embeddings = prepare_database()

In [56]:
train_images_embeddings['001_001_001']

array([[ 0.15641016,  0.05222272, -0.05158556, -0.06820939, -0.06904899,
         0.04225592, -0.26231194, -0.00162374, -0.08420499, -0.01482287,
        -0.0102939 ,  0.15204217, -0.09350223,  0.04751017,  0.0400623 ,
        -0.00481695,  0.0344524 , -0.08433454, -0.14721756, -0.03183836,
         0.07841872, -0.00340025, -0.07460167, -0.09021194,  0.01456932,
        -0.06377899,  0.09303121, -0.04148884, -0.16686536,  0.09871604,
        -0.0544215 , -0.05322067,  0.09734283,  0.02512276,  0.12048358,
        -0.01232122,  0.14697868, -0.06088248,  0.08042841,  0.09282817,
        -0.06127114,  0.00623025, -0.06747863,  0.1769361 , -0.11723821,
        -0.00656794,  0.04283603,  0.1061457 , -0.227117  , -0.10668667,
        -0.12959589, -0.02904997,  0.06846651, -0.00067627, -0.0098416 ,
        -0.18507616, -0.16815072,  0.00256527, -0.03629062,  0.00303761,
         0.00713849,  0.07191086,  0.14574409, -0.01347527,  0.03694312,
         0.00673224,  0.12085769, -0.00368221,  0.0

In [ ]:
triplet_str=[]

t1= time.time()
for i in range(1,13):
    for ja in range(1,6):
        for k in range(22,52):
             for jn in range(1,6):
                    for jp in range(1,6):
                        print (i,ja,k,jn,jp)
                        if (jp!=ja):
                            
                            if(i<10):
                                a='00'+ str(i)+'_00'+str(i)+'_00'+str(ja)
                                b='00'+ str(i) +'_00'+str(i)+'_00'+str(jp)
                                c='0'+str(k)+'_00'+str(i)+'_00'+str(jn)
                            else:
                                a='0'+str(i)+'_0'+str(i)+'_00'+str(ja)
                                b='0'+str(i)+'_0'+str(i)+'_00'+str(jp)
                                c='0'+str(k)+'_0'+str(i)+'_00'+str(jn)
                            
                            x= (a,b,c)
                            triplet_str.append(x)
                        
                    
t2=time.time()

print (t2-t1)

In [106]:
def triplet_loss(triplet_encoding, alpha = 0.3):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = triplet_encoding[0], triplet_encoding[1], triplet_encoding[2]
    
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

In [ ]:
'''final_triplet_loss=[]
for i in triplet_str:
    print (i)
    x=(train_images_embeddings[i[0]],train_images_embeddings[i[1]],train_images_embeddings[i[2]])
    loss=triplet_loss(x, alpha = 0.3)
    final_triplet_loss.append(loss)'''

In [107]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)